# 합성 테스트 데이터셋 생성

**왜 합성 테스트 데이터(Synthetic Test Dataset) 인가?**

RAG(검색 증강 생성) 증강 파이프라인의 성능을 평가하는 것은 매우 중요합니다. 

그러나 문서에서 수백 개의 QA(질문-문맥-응답) 샘플을 수동으로 생성하는 것은 시간과 노동력이 많이 소요될 수 있습니다. 또한 사람이 만든 질문은 철저한 평가에 필요한 복잡성 수준에 도달하기 어려워 궁극적으로 평가의 품질에 영향을 미칠 수 있습니다. 

합성 데이터 생성을 사용하면 데이터 집계 프로세스에서 **개발자의 시간을 90%** 까지 줄일 수 있습니다.

- RAGAS: https://docs.ragas.io/en/latest/concepts/testset_generation.html

아래의 주석을 해제한 후 실행하여 패키지를 설치 후 진행해주세요

In [1]:
# !pip install -qU ragas

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## 문서 전처리

문서를 로드 합니다.

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
# loader = PDFPlumberLoader("/home/jun/my_project/langchain_tutorial/langsmith/data/SPRI_AI_Brief_2023년12월호_F (1).pdf")

loader = PDFPlumberLoader("/home/jun/my_project/langchain_tutorial/Eval_dataset/evaldata/약관주요QA.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

15

각 문서 객체에는 `metadata` 를 통해 액세스할 수 있는 문서에 대한 추가 정보를 저장하는 데 사용할 수 있는 메타데이터 사전이 포함되어 있습니다. 

메타데이터 사전에는 `filename` 이라는 키가 포함되어 있는지 확인하세요. 

이 키는 Test datasets 생성 프로세스에서 활용될 것이므로. 메타데이터의 `filename` 속성은 동일한 문서에 속한 청크를 식별하는 데 사용됩니다. 

In [4]:
docs[0].metadata

{'source': '/home/jun/my_project/langchain_tutorial/Eval_dataset/evaldata/약관주요QA.pdf',
 'file_path': '/home/jun/my_project/langchain_tutorial/Eval_dataset/evaldata/약관주요QA.pdf',
 'page': 3,
 'total_pages': 19,
 'Producer': 'iLovePDF',
 'ModDate': 'D:20241004014222Z'}

In [5]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [6]:
docs[0].metadata

{'source': '/home/jun/my_project/langchain_tutorial/Eval_dataset/evaldata/약관주요QA.pdf',
 'file_path': '/home/jun/my_project/langchain_tutorial/Eval_dataset/evaldata/약관주요QA.pdf',
 'page': 3,
 'total_pages': 19,
 'Producer': 'iLovePDF',
 'ModDate': 'D:20241004014222Z',
 'filename': '/home/jun/my_project/langchain_tutorial/Eval_dataset/evaldata/약관주요QA.pdf'}

## 데이터셋 생성

In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


# 데이터셋 생성기
generator_llm = ChatGroq(model="llama-3.1-70b-versatile")
# 데이터셋 비평기
critic_llm = ChatGroq(model="llama-3.2-90b-text-preview")
# 문서 임베딩 & 임베딩 모델 정의
model_name = "BAAI/bge-m3"
model_kwargs = {"device":"cpu"}
encode_kwargs = {"normalize_embeddings":True}
embeddings = HuggingFaceBgeEmbeddings(
model_name=model_name, model_kwargs=model_kwargs,encode_kwargs=encode_kwargs

)

DocumentStore를 초기화합니다. 사용자 정의 LLM과 임베딩을 사용합니다.

In [8]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatGroq(model="llama-3.1-70b-versatile"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

TestSet 을 생성합니다.

In [9]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

**질문의 유형별 분포**

- simple: 간단한 질문
- reasoning: 추론이 필요한 질문
- multi_context: 여러 맥락을 고려해야 하는 질문
- conditional: 조건부 질문

In [10]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

- documents: 문서 데이터
- test_size: 생성할 질문의 수
- distributions: 질문 유형별 분포
- with_debugging_logs: 디버깅 로그 출력 여부

In [12]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs, test_size=1, distributions=distributions, with_debugging_logs=True
)

embedding nodes:   0%|          | 0/74 [00:00<?, ?it/s]

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=52d19514-c295-4f19-a9b3-f87b368a5902,id=52d19514-c295-4f19-a9b3-f87b368a5902; trace=1ec85a7d-8cec-4019-adf6-1a25eaef87a7,id=1ec85a7d-8cec-4019-adf6-1a25eaef87a7; trace=cd02970d-dcb0-46a3-9ae6-d183fc57659f,id=cd02970d-dcb0-46a3-9ae6-d183fc57659f; trace=b0514b5f-9145-45d7-a5a9-78b02e61c643,id=b0514b5f-9145-45d7-a5a9-78b02e61c643; trace=52a8ca8a-d63b-4825-a054-8ed3d1de5616,id=52a8ca8a-d63b-4825-a054-8ed3d1de5616; trace=67794b5a-3f78-46a2-9f2a-b7c56ada4180,id=67794b5a-3f78-46a2-9f2a-b7c56ada4180; trace=f3e2f97b-9c1b-41c7-9545-5b8641f27019,id=f3e2f97b-9c1b-41c7-9545-5b8641f27019; trace=ea09f96c-1032-43fd-acf0-a218e6c32ae5,id=ea09f96c-1032-43fd-acf0-a218e6c32ae5; trace=ea2f82ec-53a8-49c4

Generating:   0%|          | 0/1 [00:00<?, ?it/s]

[ragas.testset.filters.DEBUG] context scoring: {'clarity': 3, 'depth': 3, 'structure': 3, 'relevance': 3, 'score': 3.0}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['보험료의 납입', '제2회 이후의 보험료', '진단계약에서 보험료사유', '제29조(제2회 이후 보험료의 납입)', '제30조(적립보험료의 납입중지)']
[ragas.testset.evolutions.INFO] seed question generated: \uc81c29\uc870\uc758 \ub0a9\uc785\ucd5c \uc81c2\ud68c \uc774\ud6c4\uc758 \ubcf4\ud5d8\ub8cc\uc758 \ub0a9\uc785\uba74\uc81c\uc9c0\uc6d0 \uc5c6\uc2b5\ub2c8\ub2e4.
[ragas.testset.filters.DEBUG] filtered question: {'feedback': 'The question appears to be written in Korean and asks about the relationship between the performance of a specific model (\\uc81c29\\uc870\\uc758) and the characteristics of its training data. However, the question is unclear for those who do not read Korean or are unfamiliar with the specific terminology or context. For broader clarity and answerability, it would be helpful if the question was translated into English or included a brief explanati

Failed to batch ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch', '{"detail":"Monthly unique traces usage limit exceeded"}')
post: trace=4a6d7f77-329d-4e4f-a0bb-2f690ae5b644,id=4a6d7f77-329d-4e4f-a0bb-2f690ae5b644; trace=6703ce4f-a594-4ba9-ba89-ed601921919a,id=6703ce4f-a594-4ba9-ba89-ed601921919a; trace=b24e0cbe-a5b7-4892-b499-163252f89ac7,id=b24e0cbe-a5b7-4892-b499-163252f89ac7; trace=df3d2a5d-5b2c-4386-bafb-2b73fa58b181,id=df3d2a5d-5b2c-4386-bafb-2b73fa58b181; trace=c7df8cae-1089-48da-8ac1-58baf931d37b,id=c7df8cae-1089-48da-8ac1-58baf931d37b; trace=1397a695-0255-4a49-aea2-1c4c43dc5bc7,id=1397a695-0255-4a49-aea2-1c4c43dc5bc7; trace=19d5ca67-2f50-4713-a2fa-41a9eb5386ab,id=19d5ca67-2f50-4713-a2fa-41a9eb5386ab; trace=0bc8598a-b8ed-4384-80a6-928c525674db,id=0bc8598a-b8ed-4384-80a6-928c525674db; trace=dccb7dc0-5c87-48f2

In [13]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [15]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

""


DataFrame 에 저장된 데이터셋을 csv 파일로 저장합니다

In [16]:
# DataFrame의 상위 5개 행 출력
test_df.head()

""


In [17]:
# DataFrame을 CSV 파일로 저장
test_df.to_csv("/home/jun/my_project/langchain_tutorial/Eval_dataset/ragas_synthetic_dataset.csv2", index=False)

In [15]:
# from langchain_google_genai import ChatGoogleGenerativeAI


In [16]:

# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro",
#     temperature=0)

In [ ]:
r